## Part 5.3: Random Forest
Here we are going to implement a Random Forest classification algorithm. A Random Forest is composed of different decision tree predictors. For each tree, at each iteration, the data are split into two branches according to a threshold value for one of the features. At each split, the best feature and the best threshold value are chosen by minimizing the entropy function 
𝐻 given by:
$$
\displaystyle H = - \sum_{i=+1,-1} f_i \log_2(f_i)
$$
where $f_i$ is the fraction of points of class $i=+1,-1$ in the training set. The splitting continues until either a maximum depth for the tree is reached, in which case the most frequent class of the branch is assigned to all points, or the points of the branch all belong to the same class (in this case the branch is called a leaf). Different trees are built by randomly drawing data with replacement from the test set, applying the algorithm to each tree. The final label for a point is chosen based on a majority vote of the trees results. 

The hyper-parameters of the model are the **number of trees** and the **maximum depth** of the trees. We are going to implement a K-fold cross validation to choose as best hyper-parameters the ones that maximize the area under the ROC curve of the classifier. To build the ROC curve we need to estimate the probability for the points to belong to a certain class. This can be done because each tree, at the last decision step, assigns a class label based on the majority class of the training samples that reached that final leaf node, but also keeps track of the proportion of the classes in that leaf node. For example, suppose a test point ends up in a leaf node where, out of 100 training samples that ended up there, 80 belonged to class 1 and 20 belonged to class -1. The probability of the point belonging to class 1 would be 0.8, while the probability of it belonging to class -1 would be 0.2.

We will use the class `RandomForestClassifier` from `scikit-learn` to implement and train the Forest and to predict the labels with their probabilities.   


**UTILIZZIAMO COME CRITERIO L'INTERA AREA SOTTO LA CURVA O SOLO FINO AD UNA CERTA SOGLIA DI FALSE POSITIVES??**


In [22]:
import time
num_trees= np.arange(40,300,10)
max_depth= np.arange(10,40,2)
accuracies=[]
roc_areas=[]
best_area=0
best_acc=0
best_model= [] # stores best number of trees and best max depth
partial_aucs = []

start_time = time.time()

max_iter = len(num_trees) * len(max_depth)
iteration = 0

K = 5
kf = KFold(n_splits=K, shuffle=True, random_state=42) 

for tree in num_trees:
    for depth in max_depth:
        ############accuracies?
        partial_auc = 0
        auc_value = 0
        accuracy = 0
        for i, (train_index, test_index) in enumerate(kf.split(X)):
            print(f"  Fold n. {i}")
            train_K=data_rescaled_and_indexed.iloc[train_idx]
            valid_K=data_rescaled_and_indexed.iloc[val_idx]
            X_train_K, Y_train_K = split_X_Y(train_K)
            X_valid_K, Y_valid_K = split_X_Y(valid_K)
            forest = RandomForestClassifier(n_estimators=tree,  # number of trees
                                           max_depth= depth, 
                                           max_features=X_train.shape[1],   # number of features for each split (random subset)
                                           bootstrap=True,    # bagging (sampling with replacement)
                                           max_samples=1.0,   # percentage of samples to use for each tree (100% of data)
                                           random_state=10,
                                           n_jobs=-1)
            # train each model, predict labels and probability of each label
            forest.fit(X_train_K, Y_train_K)
            Y_pred = forest.predict(X_valid_K)
            y_pred_prob = forest.predict_proba(X_valid_K)[:, 1] # probability for the validation points to belong to class +1 or -1
            # accuracy on validation set
            accuracy_k = accuracy_score(Y_valid_K, Y_pred)
            iteration +=1 
            print(f'elapsed time: {round(time.time()-start_time,2)}s, done {iteration} out of {max_iter}')
            print('Number of trees:',tree,', Maximum depth',depth,':') 
            print('Accuracy:',accuracy_k)
            # false positive rates and the true positive rates for different threshold values 
            fpr, tpr, thresholds = roc_curve(Y_test, y_pred_prob)
            # area under the ROC curve on validation set 
            roc_auc_k = auc(fpr, tpr)
            roc_areas.append(roc_auc_k)
            '''if(roc_auc>best_area):
                best_area= roc_auc
                best_acc= accuracy
                best_model=[tree,depth]
                '''
            partial_fpr = fpr[fpr <= 0.2]
            if (len(partial_fpr) < 2 ):
                partial_auc = 0
            else:
                partial_tpr = tpr[:len(partial_fpr)]
                partial_auc_k = auc(partial_fpr, partial_tpr)
            partial_auc += partial_auc_k/K
            auc_value += roc_auc_k/K
        
        partial_aucs.append(partial_auc)
        if(partial_auc>best_area):
            best_area= partial_auc
            best_total_auc = roc_auc
            best_acc= accuracy
            best_model=[tree,depth]
        print('Area under ROC curve:',roc_auc,'Selected area:',partial_auc,'\n')
        
print('Best number of trees:', best_model[0],'\n')
print('Best maximum depth:', best_model[1],'\n')
print('Area under ROC curve of best model:',best_area)

  Fold n. 0
elapsed time: 0.44s, done 1 out of 390
Number of trees: 40 , Maximum depth 10 :
Accuracy: 0.8125164343938995
  Fold n. 1
elapsed time: 0.87s, done 2 out of 390
Number of trees: 40 , Maximum depth 10 :
Accuracy: 0.8125164343938995
  Fold n. 2
elapsed time: 1.27s, done 3 out of 390
Number of trees: 40 , Maximum depth 10 :
Accuracy: 0.8125164343938995
  Fold n. 3
elapsed time: 1.66s, done 4 out of 390
Number of trees: 40 , Maximum depth 10 :
Accuracy: 0.8125164343938995
  Fold n. 4
elapsed time: 2.03s, done 5 out of 390
Number of trees: 40 , Maximum depth 10 :
Accuracy: 0.8125164343938995
Area under ROC curve: 0.8638111382393521 Selected area: 0.09767014902808546 

  Fold n. 0
elapsed time: 2.47s, done 6 out of 390
Number of trees: 40 , Maximum depth 12 :
Accuracy: 0.8161977386273994
  Fold n. 1
elapsed time: 2.9s, done 7 out of 390
Number of trees: 40 , Maximum depth 12 :
Accuracy: 0.8161977386273994
  Fold n. 2
elapsed time: 3.31s, done 8 out of 390
Number of trees: 40 , Max

KeyboardInterrupt: 

In [ ]:
print("Best partial AUC", best_area)
print("Best total AUC", best_total_auc)


Now we train again the best Random Forest found on the whole training set. We calculate its accuracy and $Q$ factor on the test set, and find the relative ROC curve of the classifier.  

In [ ]:
X_train_np = np.array(X_train)
Y_train_np = np.array(Y_train)
X_test_np = np.array(X_test)
Y_test_np = np.array(Y_test)

best_forest = RandomForestClassifier(n_estimators=best_model[0], max_depth= best_model[1], max_features=X_train_np.shape[1],bootstrap=True,max_samples=1.0,random_state=10,n_jobs=-1)

best_forest.fit(X_train_np,Y_train_np)
Y_pred = best_forest.predict(X_test_np)
accuracy = accuracy_score(Y_test_np, Y_pred)
print('Accuracy of best Random Forest:',accuracy)

y_pred_prob = best_forest.predict_proba(X_test_np)[:, 1]
fpr, tpr, thresholds = roc_curve(Y_test_np, y_pred_prob)
roc_auc = auc(fpr, tpr)
partial_fpr = fpr[fpr <= 0.2]
partial_tpr = tpr[:len(partial_fpr)]
partial_auc = auc(partial_fpr, partial_tpr)
roc_auc_best = auc(fpr, tpr)
print('Partial AUC of the best Random Forest:',partial_auc)
print('AUC of the best Random Forest',roc_auc)

In [ ]:
Q_rforest = get_best_Q(fpr,tpr)
fpr_rforest = fpr
tpr_rforest = tpr

print('best random forest Q:',Q_rforest)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', lw=1.5, label=f'ROC curve (AUC = {roc_auc_best:.3f})')
plt.plot([0, 1], [0, 1], color='red', linestyle='--') 
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.02])
plt.xlabel('False Positive Rate',fontsize=13.8)
plt.ylabel('True Positive Rate',fontsize=13.8)
plt.title('Random Forest ROC Curve',fontsize=14.3)
plt.grid(alpha=0.4)
plt.legend(loc='best')
plt.show()